In [5]:
import pandas as pd
import numpy as np

In [6]:
from comment_parser import comment_parser

In [17]:
filename = "../data/NL2ML_ structure - levin.csv"

df = pd.read_csv(filename, skiprows=1)

In [18]:
df.head(5)

,chunk_id,notebook_id,splitting_id,code_block,data_format,graph_vertex,errors,graph_vertex_m1,graph_vertex_m2,graph_vertex_m3,...,python_methods_m3,python_methods_p1,python_methods_p2,python_methods_p3,kaggle_link,kaggle_comments,kaggle_upvotes,kaggle_section,kaggle_section_overview,kaggle_score
0,1.0,1.0,1.0,import os\nimport numpy as np\nimport matplotl...,None,Environment.import_modules,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1.0,1.0,"print('Reading datasets...', end='')\n\ntrain_...",Table,Data: extraction.Load from CSV,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,1.0,1.0,"print('Preprocessing data...', end='')\n\nfeat...",Table,Data: transform.filter;Data: transform.data ty...,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,1.0,1.0,"print('Creating classifier...', end='')\nclf =...",None,Model: train.choose model class,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,1.0,1.0,"print('Training classifier...', end='')\r\n%ti...",Table,Model: train.train model,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save the column names from the original markup file

In [19]:
# HEADER = df.columns
# HEADER = [col if ("Unnamed:" not in col) else "" for col in HEADER ]

In [20]:
# df.columns = df.iloc[0,:]

In [21]:
# df = df.drop(index=0)

In [22]:
# df.head(5)

### Functions for parsing libraries and comments

In [23]:
def get_dict(str_arr):
    name_lib_import = {}
    name_lib_from = {}
    str_arr = [[line.strip() for line in text.split("\n")] for text in str_arr]
    # delete all double spaces in all lines
    for line_array in str_arr:
        for ind in range(len(line_array)):
            prev_len = -1
            while prev_len != len(line_array[ind]):
                prev_len = len(line_array[ind])
                line_array[ind] = line_array[ind].replace("  ", " ")
    # now get libraries if line start with "import" or "from"
    for line_array in str_arr:
        for line in line_array:
            if line.startswith("import"):
                split_arr = line.split(" ")
                # if we have "import ... as ...""
                if len(split_arr) > 2 and split_arr[2] == "as":
                    name_lib_import[split_arr[-1]] = split_arr[1].split(".")[0]
            
            if line.startswith("from"):
                split_arr = line.split(" ")
                # if we have "from ... import ...""
                if len(split_arr) > 2 and split_arr[2] == "import":
                    name_lib_from[split_arr[-1]] = split_arr[1].split(".")[0]
    return (name_lib_import, name_lib_from)
    
    
def get_libraries(text: str, dicts: tuple):
    name_lib_import = dicts[0]
    name_lib_from = dicts[1]
    text = str(text)
    libs = []
    line_array = [line.strip() for line in text.split("\n")]
    # delete all double spaces in all lines
    for ind in range(len(line_array)):
        
        prev_len = -1
        while prev_len != len(line_array[ind]):
            prev_len = len(line_array[ind])
            line_array[ind] = line_array[ind].replace("  ", " ")
    # now get libraries if line start with "import" or "from"
    for line in line_array:
        if line.startswith("import") or line.startswith("from"):
            libs.append(line.split(" ")[1].split(".")[0])
    for name in name_lib_from.keys():
        for line in line_array:
            if name in line:
                libs.append(name_lib_from[name])
                
    for name in name_lib_import.keys():
        for line in line_array:
            if name + "." in line:
                libs.append(name_lib_import[name])
        
    return "\n".join(list(set(libs)))


def get_comments(text: str):
    text = str(text)
#     comments = comment_parser.extract_comments_from_str(text, mime="text/x-python")
#     return "\n".join([line.text() for line in comments])

    comments = []
    line_array = [line.strip() for line in text.split("\n")]
    # now get libraries if line start with "#"
    for line_ind in range(len(line_array)):
        if line_array[line_ind].startswith("#"):
            comments.append(line_array[line_ind][1:])
        elif line_array[line_ind].startswith("'''"):
            multi_comm = str()
            multi_comm += line_array[line_ind][3:] + "\n"
            if "'''" in multi_comm:
                multi_comm = multi_comm.replace("'''", "")
                line_ind += 1
                comments.append(multi_comm)
                continue
            line_ind += 1
            while line_ind < len(line_array):
                multi_comm += line_array[line_ind] + "\n"
                if "'''" in multi_comm:
                    multi_comm = multi_comm.replace("'''", "")
                    line_ind += 1
                    break
                line_ind += 1
            comments.append(multi_comm)
        elif line_array[line_ind].startswith('"""'):
            multi_comm = str()
            multi_comm += line_array[line_ind][3:] + "\n"
            if '"""' in multi_comm:
                multi_comm = multi_comm.replace('"""', "")
                line_ind += 1
                comments.append(multi_comm)
                continue
            line_ind += 1
            while line_ind < len(line_array):
                multi_comm += line_array[line_ind] + "\n"
                if '"""' in multi_comm:
                    multi_comm = multi_comm.replace('"""', "")
                    line_ind += 1
                    break
                line_ind += 1
            comments.append(multi_comm)
            pass
    return "\n".join([line for line in comments])

In [24]:
# df['libraries'] = df.code_block.apply(get_libraries)

In [25]:
df['comments'] = df.code_block.apply(get_comments)

In [26]:
df['comments']

0                                                       
1                                                       
2                           train = train.fillna(-99999)
3      sampling_method='gradient_based',\neval_metric...
4                                                       
                             ...                        
994                                                     
995                                                     
996                                                     
997                                                     
998                                                     
Name: comments, Length: 999, dtype: object

### For each 'notebook_id' do cell shift

In [27]:
notebooks_ids = set([i for i in df['notebook_id'].values if str(i) != 'nan'])
SHIFT_RANGE = 3
all_temp_dfs = []

# print(notebooks_ids)
for not_id in notebooks_ids:
    # get rows for one notebook_id
    temp_df = df[df['notebook_id'] == not_id]
    buf_graph_vertex = list(temp_df.graph_vertex.values)
    dicts = get_dict(temp_df.code_block.values)
#     print(dicts)
    buf_arr = []
    for code in temp_df.code_block.values:
#         print(get_libraries(code, dicts))
        buf_arr.append(get_libraries(code, dicts))
    temp_df['libraries'] = buf_arr
    # shift down
    for i in range(1, SHIFT_RANGE + 1):
        temp_df['graph_vertex_m' + str(i)] = [np.NaN] * i + buf_graph_vertex[0: -i]
    # shift up
    for i in range(1, SHIFT_RANGE + 1):
        temp_df['graph_vertex_p' + str(i)] = buf_graph_vertex[i:] + [np.NaN] * i
    all_temp_dfs.append(temp_df)
    
# concatenate all shifted notebooks
final_df = pd.concat(all_temp_dfs)
final_df['notebook_id'] = final_df['notebook_id'].apply(int)
final_df['chunk_id'] = final_df['chunk_id'].apply(int)

final_df.sort_values(['notebook_id', 'chunk_id'], inplace=True)
final_df.head()

C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,chunk_id,notebook_id,splitting_id,code_block,data_format,graph_vertex,errors,graph_vertex_m1,graph_vertex_m2,graph_vertex_m3,...,python_methods_m3,python_methods_p1,python_methods_p2,python_methods_p3,kaggle_link,kaggle_comments,kaggle_upvotes,kaggle_section,kaggle_section_overview,kaggle_score
0,1,1,1.0,import os\nimport numpy as np\nimport matplotl...,None,Environment.import_modules,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1.0,"print('Reading datasets...', end='')\n\ntrain_...",Table,Data: extraction.Load from CSV,False,Environment.import_modules,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1.0,"print('Preprocessing data...', end='')\n\nfeat...",Table,Data: transform.filter;Data: transform.data ty...,False,Data: extraction.Load from CSV,Environment.import_modules,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,1.0,"print('Creating classifier...', end='')\nclf =...",None,Model: train.choose model class,False,Data: transform.filter;Data: transform.data ty...,Data: extraction.Load from CSV,Environment.import_modules,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,1.0,"print('Training classifier...', end='')\r\n%ti...",Table,Model: train.train model,False,Model: train.choose model class,Data: transform.filter;Data: transform.data ty...,Data: extraction.Load from CSV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
final_df[['code_block', 'libraries']]

,code_block,libraries
0,"import os\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nimport cudf\n\nfrom sklearn import preprocessing\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import StratifiedKFold\n\nimport pandas as pd\npd.set_option('display.max_columns', 500)\n\nimport xgboost as xgb\nprint(""XGBoost version:"", xgb.__version__)\n\nimport warnings\nwarnings.filterwarnings(""ignore"")\n\nimport janestreet\nprint('Creating competition environment...', end='')\nenv = janestr...",xgboost\nmatplotlib\npandas\ncudf\nos\njanestreet\nsklearn\nwarnings\nnumpy
1,"print('Reading datasets...', end='')\n\ntrain_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')\ntrain = train_cudf.to_pandas()\ndel train_cudf\n\nfeatures_meta_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')\nfeatures_meta = features_meta_cudf.to_pandas()\ndel features_meta_cudf\n\nprint('Finished.')\n\nprint(f'train shape: {format(train.shape)}')\nprint(f'features_meta shape: {format(features_meta.shape)}')",
2,"print('Preprocessing data...', end='')\n\nfeatures = [col for col in list(train.columns) if 'feature' in col]\n\ntrain = train[train['weight'] != 0]\n\ntrain['action'] = (train['resp'].values > 0).astype(int)\n\n#train = train.fillna(-99999)\ntrain.fillna(f_mean)\n\nX_train = train.loc[:, features]\ny_train = train.loc[:, 'action']\ndel train\n\nprint('Finished.')",
3,"print('Creating classifier...', end='')\nclf = xgb.XGBClassifier(\n n_estimators=400,\n max_depth=7,\n eta=0.5, # learning_rate\n missing=None,\n random_state=42,\n tree_method='gpu_hist',\n subsample=0.8,\n colsample_bytree=1,\n #sampling_method='gradient_based',\n #eval_metric='logloss',\n verbosity=2 # info\n)\nprint('Finished.')",xgboost
4,"print('Training classifier...', end='')\r\n%time clf.fit(X_train, y_train)\r\nprint('Finished.')",
5,"print('Creating submissions file...', end='')\nrcount = 0\nfor (test_df, prediction_df) in env.iter_test():\n X_test = test_df.loc[:, features]\n y_preds = clf.predict(X_test)\n prediction_df.action = y_preds\n env.predict(prediction_df)\n rcount += len(test_df.index)\nprint(f'Finished processing {rcount} rows.')",
6,import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.linear_model import Lasso\nfrom sklearn.linear_model import ElasticNet\nfrom sklearn.ensemble import RandomForestRegressor\nimport xgboost\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.model_selection import GridSearchCV\nimport warnings\nwarnings.filterwarnings('ignore'),xgboost\nmatplotlib\npandas\nseaborn\nsklearn\nwarnings\nnumpy
7,train=pd.read_csv(r'../input/house-prices-advanced-regression-techniques/train.csv'),pandas
8,train.head()\r,
9,train.shape,


### Returning the original header markup

In [39]:
# add columns names 
final_df.loc[-1] = final_df.columns
# add original HEADER
final_df.columns = HEADER
final_df.index = final_df.index + 1
final_df.sort_index(inplace=True)

In [41]:
final_df[['code_block', 'graph_vertex'.head(25)

,PK,Manually,,,,,,Automatically,,,...,,,,,,,,,,
0,chunk_id,notebook_id,splitting_id,code_block,data_format,graph_vertex,errors,graph_vertex_m1,graph_vertex_m2,graph_vertex_m3,...,python_methods_m3,python_methods_p1,python_methods_p2,python_methods_p3,kaggle_link,kaggle_comments,kaggle_upvotes,kaggle_section,kaggle_section_overview,kaggle_score
1,1,1,1,"import os\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nimport cudf\n\nfrom sklearn import preprocessing\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import StratifiedKFold\n\nimport pandas as pd\npd.set_option('display.max_columns', 500)\n\nimport xgboost as xgb\nprint(""XGBoost version:"", xgb.__version__)\n\nimport warnings\nwarnings.filterwarnings(""ignore"")\n\nimport janestreet\nprint('Creating competition environment...', end='')\nenv = janestr...",None,Environment.import_modules,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,1,"print('Reading datasets...', end='')\n\ntrain_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')\ntrain = train_cudf.to_pandas()\ndel train_cudf\n\nfeatures_meta_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')\nfeatures_meta = features_meta_cudf.to_pandas()\ndel features_meta_cudf\n\nprint('Finished.')\n\nprint(f'train shape: {format(train.shape)}')\nprint(f'features_meta shape: {format(features_meta.shape)}')",Table,Data: extraction.Load from CSV,False,Environment.import_modules,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,1,"print('Preprocessing data...', end='')\n\nfeatures = [col for col in list(train.columns) if 'feature' in col]\n\ntrain = train[train['weight'] != 0]\n\ntrain['action'] = (train['resp'].values > 0).astype(int)\n\n#train = train.fillna(-99999)\ntrain.fillna(f_mean)\n\nX_train = train.loc[:, features]\ny_train = train.loc[:, 'action']\ndel train\n\nprint('Finished.')",Table,Data: transform.filter;Data: transform.data type conversions;Data: transform.correct missing values;Data: transform.split,False,Data: extraction.Load from CSV,Environment.import_modules,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,1,"print('Creating classifier...', end='')\nclf = xgb.XGBClassifier(\n n_estimators=400,\n max_depth=7,\n eta=0.5, # learning_rate\n missing=None,\n random_state=42,\n tree_method='gpu_hist',\n subsample=0.8,\n colsample_bytree=1,\n #sampling_method='gradient_based',\n #eval_metric='logloss',\n verbosity=2 # info\n)\nprint('Finished.')",None,Model: train.choose model class,False,Data: transform.filter;Data: transform.data type conversions;Data: transform.correct missing values;Data: transform.split,Data: extraction.Load from CSV,Environment.import_modules,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1,1,"print('Training classifier...', end='')\r\n%time clf.fit(X_train, y_train)\r\nprint('Finished.')",Table,Model: train.train model,False,Model: train.choose model class,Data: transform.filter;Data: transform.data type conversions;Data: transform.correct missing values;Data: transform.split,Data: extraction.Load from CSV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,1,1,"print('Creating submissions file...', end='')\nrcount = 0\nfor (test_df, prediction_df) in env.iter_test():\n X_test = test_df.loc[:, features]\n y_preds = clf.predict(X_test)\n prediction_df.action = y_preds\n env.predict(prediction_df)\n rcount += len(test_df.index)\nprint(f'Finished processing {rcount} rows.')",Table,Model: train.predict,False,Model: train.train model,Model: train.choose model class,Data: transform.filter;Data: transform.data type conversions;Data: transform.correct missing values;Data: transform.split,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7,1,import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.linear_model import Lasso\nfrom sklearn.linear_model import ElasticNet\nfrom sklearn.ensemble import RandomForestRegressor\nimport xgboost\

In [ ]:
final_df.to_csv(filename[:-4] + "graph_vertex_shift.csv", index=False)